<a href="https://colab.research.google.com/github/Quooooooookka/Q-learning-Tic-Tac-Toe/blob/master/Instance_Recognition_(GPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Change working directory to root
*   Clone matterport Mask_RCNN GitHub repo into Google Colab

In [1]:
%cd

!git clone --quiet https://github.com/matterport/Mask_RCNN.git

/root




*   Install wrapper library of google-api-python-client
*   Install requirements from requirements.txt
*   Install setup.py







In [2]:
%cd ~/Mask_RCNN

!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

/root/Mask_RCNN
     |████████████████████████████████| 163kB 7.2MB/s 
     |████████████████████████████████| 204kB 11.4MB/s 
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links t

*   Change working directory to Mask_RCNN
*   Retrieve dataset from Google Drive
*   Unzip and unpack dataset

In [3]:
%cd ~/Mask_RCNN

fileId = '1Wd3vxm70PCYh7hY5K1AaYeHdW1S7XK-G'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

os.makedirs('dataset')
os.chdir('dataset')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)

os.system('mv /root/Mask_RCNN/dataset/images/val /root/Mask_RCNN/dataset')
os.system('mv /root/Mask_RCNN/dataset/images/train /root/Mask_RCNN/dataset')
os.rmdir('/root/Mask_RCNN/dataset/images')

print('[+] Extracted zip file: ' + fileName)

/root/Mask_RCNN
[+] Extracted zip file: 1Wd3vxm70PCYh7hY5K1AaYeHdW1S7XK-G.zip




*   Change working directory to Mask_RCNN
*   Set class to car.py
*   Set epochs hyperparameter (to 1 for test purposes)



In [4]:
%cd ~/Mask_RCNN

!cp ~/Mask_RCNN/samples/balloon/balloon.py ./car.py

!sed -i -- 's/balloon/car/g' car.py
!sed -i -- 's/Balloon/Car/g' car.py
!sed -i -- 's/epochs=30/epochs=1/g' car.py

/root/Mask_RCNN


*   Uninstall incompatible keras and tensorflow framework
*   Install compatible keras and tensorflow framework

In [5]:
!pip3 uninstall keras-nightly
!pip3 uninstall -y tensorflow
!pip3 install keras==2.1.6
!pip3 install tensorflow==1.15.0
!pip3 install h5py==2.10.0

Uninstalling keras-nightly-2.5.0.dev2021032900:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras/*
    /usr/local/lib/python3.7/dist-packages/keras_nightly-2.5.0.dev2021032900.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/keras/applications/resnet50.py
    /usr/local/lib/python3.7/dist-packages/keras/engine/network.py
    /usr/local/lib/python3.7/dist-packages/keras/engine/topology.py
    /usr/local/lib/python3.7/dist-packages/keras/initializers.py
    /usr/local/lib/python3.7/dist-packages/keras/layers/experimental/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/layers/experimental/preprocessing/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/objectives.py
    /usr/local/lib/python3.7/dist-packages/keras/optimizers/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/optimizers/schedules/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/utils/test_utils.py
Proceed (y/n)?

*   Change working directory to Mask_RCNN
*   Train model on custom dataset with transfer learning (COCO weights)



In [6]:
%cd ~/Mask_RCNN

!python car.py train --dataset=dataset/ --weights=coco

/root/Mask_RCNN
Using TensorFlow backend.
Weights:  coco
Dataset:  dataset/
Logs:  /logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_

*   Import Mask RCNN
*   Run detection one image at a time
*   Load validation dataset
*   Create model in inference mode
*   Load latest trained model
*   Load weights







In [12]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import car

ROOT_DIR = os.getcwd()

sys.path.append(ROOT_DIR)

custom_WEIGHTS_PATH = sorted(glob.glob("/logs/*/mask_rcnn_*.h5"))[-1]

%matplotlib inline 

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

config = car.CarConfig()
custom_DIR = os.path.join(ROOT_DIR, "dataset")

class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

DEVICE = "/gpu:0"
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
  
dataset = car.CarDataset()
dataset.load_car(custom_DIR, "val")
dataset.prepare()

print(f'Images: {len(dataset.image_ids)}\nClasses: {dataset.class_names}')

with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

print(f'[+] Loading weights, {custom_WEIGHTS_PATH}')
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload
reload(visualize)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

<module 'mrcnn.visualize' from '/root/Mask_RCNN/mrcnn/visualize.py'>

*   Iterate through images
*   Print image information
*   Feedforward dataset/val images
*   Calculate and print mean mask size (px^2)



In [13]:
for image_id in dataset.image_ids:
  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
      modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
  info = dataset.image_info[image_id]
  
  print(f'[+] Image ID: {info["source"]}.{info["id"]} ({image_id}) {dataset.image_reference(image_id)}')

  results = model.detect([image], verbose=0)[0]
  class_ids = results['class_ids']
  mask = results['masks']
  unique_class_ids = np.unique(class_ids)
  mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]]) for i in unique_class_ids]
  area = mask_area[0] / len(class_ids)

  print(f'    Average area: {area}\n')


[+] Image ID: car.1.jpg (0) /root/Mask_RCNN/dataset/val/1.jpg
    Average area: 179844.0

[+] Image ID: car.2.jpg (1) /root/Mask_RCNN/dataset/val/2.jpg
    Average area: 229934.0

[+] Image ID: car.3.jpg (2) /root/Mask_RCNN/dataset/val/3.jpg
    Average area: 179732.0

[+] Image ID: car.4.jpg (3) /root/Mask_RCNN/dataset/val/4.jpg
    Average area: 140352.0

[+] Image ID: car.5.jpg (4) /root/Mask_RCNN/dataset/val/5.jpg
    Average area: 278056.0

[+] Image ID: car.6.jpg (5) /root/Mask_RCNN/dataset/val/6.jpg
    Average area: 198430.0

[+] Image ID: car.7.jpg (6) /root/Mask_RCNN/dataset/val/7.jpg
    Average area: 36849.8

[+] Image ID: car.8.png (7) /root/Mask_RCNN/dataset/val/8.png
    Average area: 229780.0

